In [1]:
import spacy
nlp = spacy.load('en_core_web_sm')

In [2]:
import language_tool_python

In [3]:
# def correct_grammar(string):
#     tool = LanguageTool('en-US')
#     matches = tool.check(string)
#     return tool.correct(string, matches)
def correct_grammar(string):
    tool = language_tool_python.LanguageTool('en-US')
    print("Correct")
    matches = tool.check(string)
    return language_tool_python.utils.correct(string, matches)

In [4]:
import textacy
from textacy import preprocessing

In [5]:
from spacy.matcher import Matcher

In [6]:
def separate_punc(doc_text):
    return [token.text.lower() for token in nlp(doc_text) if token.text not in '\n\n \n\n\n!"-#$%&()--.*+,-/:;<=>?@[\\]^_`{|}~\t\n ']

In [7]:
def read_file(filepath):
    
    with open(filepath) as f:
        str_text = f.read()
    
    return str_text

In [8]:
def print_list_string(lst):
    for i in range(len(lst)):
        print(i+1,":",lst[i])
        print()

In [9]:
def detect_past_sentence(sentence):
    sent = list(nlp(sentence).sents)[0]
    return (
        sent.root.tag_ == "VBD" or
        any(w.dep_ == "aux" and w.tag_ == "VBD" for w in sent.root.children))

In [10]:
def detect_passive_voice(sentence):
    matcher = Matcher(nlp.vocab)
    doc = nlp(sentence)    
    passive_rule = [{'DEP':'nsubjpass'},{'DEP':'aux','OP':'*'},{'DEP':'auxpass'},{'TAG':'VBN'}]
    matcher.add('Passive',[passive_rule])
    matches = matcher(doc)
    if len(matches) > 0:
        return True
    else:
        return False

In [11]:
def identify_actors(desc):
    doc1 = nlp(desc)
    actor = []
    for sent in doc1.sents:
        subj = ""
        try:
            SBOV_ext = textacy.extract.subject_verb_object_triples(sent)
            SBOV = list(SBOV_ext)
            if SBOV:
                for triple in SBOV:
                    subj_tokens = triple[0]  # Subject tokens
                    subj_text = ' '.join(token.text for token in subj_tokens)
                    actor.append(subj_text)
        except Exception as e:
            print(f"Error occurred: {e}")

        for token in sent:
            #--------------------------------actor identification--------------------------------
            if token.tag_ == "NN" and token.dep_ == "nsubj" and subj == token.text:
                if token.text.lower() not in actor:
                    actor.append(token.text.lower())
    return actor
    

In [12]:
def identify_nsubj(sent):
    for token in sent:
        if token.dep_ == "nsubj":
            return token.text.lower()
    return None

In [13]:
def make_EARS_template_1(sent,EARS_template_1):         #WHEN ..system shall...
    temp = list.copy(EARS_template_1)
    
    comma_idx = sent.text.find(",")
    
    temp[1] = sent.text[5:comma_idx]
    
    comma_flg = False
    rem_str = ""
    verb_flg = False
    
    for token in sent:
        if token.tag_ == ",":
            comma_flg = True
        elif comma_flg:
            if token.pos_ == "VERB":
                temp[3] += token.lemma_
                temp[3] += " "
                verb_flg = True
            elif verb_flg:
                if token.text.lower() == "system":
                    continue
                else:
                    rem_str += token.text
                    rem_str += " "
    
    temp[3] += rem_str
    
    if temp[1] != "" and temp[3] != "":
        return correct_grammar("".join(temp))
    else:
        return ""
    


In [14]:
def make_EARS_template_2(sent,EARS_template_2,actor):
    temp = list.copy(EARS_template_2)
        

    then_idx = sent.text.find("then")
    then_next = then_idx+5

    if then_idx == -1:             #if then not found
        then_idx = sent.text.find(",")
        then_next = then_idx+1

    if sent.text.startswith("If"):    
        temp[1] = sent.text[3:then_idx]
    else:
        temp[1] = sent.text[:then_idx]


    verb_string = str("")
    temp[3] = ""
    idx_rest_of_sent = 0
    idx_aux = 0
    flg = False
    first_flg = False
    
    #without then statements remaining
    
    for i in range(len(sent)):
        if sent[i].text == "then" and sent[i].dep_ == "advmod":
            temp[3] += str(sent[i].head.lemma_)
            verb_string = sent[i].head.text
            flg = True
            
        elif flg:
            if sent[i].pos_.lower() == "aux" and first_flg == False:
                first_flg = True
                
            elif sent[i].text == "system":
                continue
            
            else:
                if sent[i].text != verb_string:
                    if sent[i].tag_ == "PRP":
                        if actor[0] != "system":
                            temp[3] += str(actor[0])
                            temp[3] += " "
                        else:
                            temp[3] += str(actor[1])
                            temp[3] += " "
                    else:
                        temp[3] += str(sent[i].text)
                        temp[3] +=" "
        
            
    if temp[1] == "" or temp[3] == "":
        return ""
    else:
        return correct_grammar("".join(temp))
        

In [15]:
def make_EARS_template_3(sent,EARS_template_3):
    temp = list.copy(EARS_template_3)    #while template
    
    comma_idx = sent.text.find(",")
    
    temp[1] = sent.text[6:comma_idx]
    
    comma_flg = False
    rem_str = ""
    verb_flg = False
    
    for token in sent:
        if token.tag_ == ",":
            comma_flg = True
        elif comma_flg:
            if token.pos_ == "VERB":
                temp[3] += token.lemma_
                temp[3] += " "
                verb_flg = True
            elif verb_flg:
                if token.text.lower() == "system":
                    continue
                else:
                    rem_str += token.text
                    rem_str += " "
    
    temp[3] += rem_str
    
    if temp[1] != "" and temp[3] != "":
        return correct_grammar("".join(temp))
    else:
        return ""

In [16]:
def make_Rupp_template_2(sent,Rupp_template_2,actor):
    temp = list.copy(Rupp_template_2)      #change index 1-customer 3-PROCESS
    subj_flg = False
    verb_flg = False
    for token in sent:
        if token.dep_ == "nsubj" and token.text.lower() in actor:
            subj_flg = True
            temp[1] = token.text
        
        elif subj_flg == True:
            if verb_flg == False and token.pos_.lower() == "verb":
                verb_flg = True
                temp[3] += token.lemma_
                temp[3] += " "
            else:
                if token.tag_ == "PRP" and (token.text == "he" or token.text == "she"):
                    if actor[0] != "system":
                        temp[3] += str(actor[0])
                        temp[3] += " "
                    else:
                        temp[3] += str(actor[1])
                        temp[3] += " "
                else:
                    temp[3] += token.text
                    temp[3] += " "
                
    if temp[1] != "" and temp[3] != "":
        return correct_grammar("".join(temp))
    else:
        return ""
    

In [17]:
def make_Rupp_template_3(sent,Rupp_template_3,actor):
    temp = list.copy(Rupp_template_3)      #change index 1-PROCESS
    subj_flg = False
    verb_flg = False
    for token in sent:
        if token.dep_ == "nsubj" and token.text.lower() in actor:
            subj_flg = True
        
        elif subj_flg == True:
            if verb_flg == False and token.pos_.lower() == "verb":
                verb_flg = True
                temp[1] += token.lemma_
                temp[1] += " "
            else:
                temp[1] += token.text
                temp[1] += " "
    
    if temp[1] != "":
        return correct_grammar("".join(temp))
    else:
        return ""
    

In [18]:
from textacy import preprocessing
# from textacy.preprocessing import normalize_whitespace, preprocess_text
def convert_requirements(desc,req):
    # req = preprocessing.normalize_whitespace(req) #removing extra spaces
    req = preprocessing.normalize.whitespace(req)
    doc = nlp(req)

    pre_cond = []
    actor_action = []
    condition = []
    response = []

    # desc = preprocessing.normalize_whitespace(desc)
    desc = preprocessing.normalize.whitespace(desc)

    actor = identify_actors(desc)

    for sent in doc.sents:
        md_flag = False
        admod = False
        mark = False
        cond = False
        sys_flg = False


        for token in sent:

            #---------------------------------actor action---------------------------------------
            if token.text in actor and detect_passive_voice(sent.text) == False and detect_past_sentence(sent.text) == False and sent not in actor_action:
                actor_action.append(sent)

            #--------------------------------precondition------------------------------------------    
            if token.tag_ == "MD":
                md_flag = True

            if md_flag == True and sent not in pre_cond:
                if detect_past_sentence(sent.text) == True:
                    if token.tag_ == "VBN":
                        pre_cond.append(sent)
                else:
                    pre_cond.append(sent)

            #------------------------------conditional-----------------------------------------------
            if token.dep_ == "advmod":
                admod = True

            if token.dep_ == "mark":
                mark = True

            if cond == False:
                if admod == True and mark == True:
                    condition.append(sent)
                    cond = True

            #------------------------------response-------------------------------------------------
            if token.text.lower() == "system":
                sys_flg = True

            if sys_flg == True and token.text.lower() in actor and detect_past_sentence(sent.text) == False and sent not in response:
                response.append(sent)


    print("---------------ACTORS--------------------")
    print_list_string(actor)
    print("---------------PRE-CONDITION--------------------")
    print_list_string(pre_cond)
    print("---------------CONDITION--------------------")
    print_list_string(condition)
    print("---------------RESPONSE--------------------")
    print_list_string(response)
    print("---------------ACTOR_ACTION--------------------")
    print_list_string(actor_action)

    #-----------------------------template structures--------------------------------
    Rupp_template_1 = ["The System shall ",""]                               #replace index 1(PROCESS)
    Rupp_template_2 = ["The System shall provide ","","the ability to ",""]  #replace index 1(ACTOR) and 3(PROCESS)
    Rupp_template_3 = ["The System shall be able to ",""]                    #replace index 1(PROCESS)

    EARS_template_1 = ["When ","","the system shall ",""]                     #replace index 1(PRE-COND||TRIGGER) and 3(SYSTEM RESPONSE)
    EARS_template_2 = ["If ","","then the system shall ",""]                  #replace index 1(PRE-COND||TRIGGER) and 3(SYSTEM RESPONSE)
    EARS_template_3 = ["While ","","the system shall ",""]                    #replace index 1(SYSTEM STATE) and 3(SYSTEM RESPONSE)
    EARS_template_4 = ["Where ","","the system shall ",""]                    #replace index 1(FEATURE) and 3(SYSTEM RESPONSE)

    #------------------------second pass-----------------------
    results = []
    start_char = []
    end_char = []

    for sent in doc.sents:
        #--------------conditoinal--------------
        if sent in condition:
            res = make_EARS_template_2(sent,EARS_template_2,actor)
            if res != "":
                results.append(res)
                start_char.append(sent.start_char)
                end_char.append(sent.end_char)

        #--------------actor_actional---------------
        elif sent in actor_action:
            if identify_nsubj(sent) == "system":
                res = make_Rupp_template_3(sent,Rupp_template_3,actor) 
                if res != "":
                    results.append(res)
                    start_char.append(sent.start_char)
                    end_char.append(sent.end_char)
            else:
                res = make_Rupp_template_2(sent,Rupp_template_2,actor)
                if res != "":
                    results.append(res)
                    start_char.append(sent.start_char)
                    end_char.append(sent.end_char)

        else:
            if sent.text.startswith("When"):
                res = make_EARS_template_1(sent,EARS_template_1)
                if res != "":
                    results.append(res)
                    start_char.append(sent.start_char)
                    end_char.append(sent.end_char)
            elif sent.text.startswith("While"):
                res = make_EARS_template_3(sent,EARS_template_3)
                if res != "":
                    results.append(res)
                    start_char.append(sent.start_char)
                    end_char.append(sent.end_char)

    
    return results, start_char, end_char


In [19]:
#--------------Quality-----------------

In [20]:
import nltk

import string
import numpy as np

from nltk.corpus import stopwords

from nltk.cluster.util import cosine_distance


from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
stop_words = set(stopwords.words("english"))
# import tokenizations

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\u25o52\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\u25o52\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\u25o52\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [21]:
def read_file(filepath):
    
    with open(filepath,encoding="utf8") as f:
        str_text = f.read()
    
    return str_text

def sentence_similarity(sent1,sent2):
    stop_words = stopwords.words('english')
    sent1 = [token.lower() for token in sent1]
    sent2 = [token.lower() for token in sent2]
    
    all_words = list(set(sent1+sent2))
    
    vector1 = [0]*len(all_words)
    vector2 = [0]*len(all_words)
    
    for w in sent1:
        if w in stop_words:
            continue
        vector1[all_words.index(w)]+=1
    
    for w in sent2:
        if w in stop_words:
            continue
        vector2[all_words.index(w)]+=1
    return 1 - cosine_distance(vector1,vector2)
    

def get_duplication_index(sentences):
    count = 0
    global len_sent
    for i in range(len(sentences)):
        for j in range(i+1,len(sentences)):
            if sentence_similarity(sentences[i],sentences[j]) > 0.99:
                count+=1
                print("Number :",count)
                print("Sent_index",i,":",sentences[i])
                print("Sent_index",j,":",sentences[j])
                
    return count/len_sent
#     print("Duplication index :"+"{:.2f}".format(count/len(sentences)))

def get_wordliness_index(requirement_text,total_sentences):
    count = 0
    tokens = nltk.word_tokenize(requirement_text)
    tagged = nltk.pos_tag(tokens)
    for i in tagged:
        if i[1] == 'RB':
            if i[0] != 'then' and i[0] != 'not' and i[0] != 'also':
                print(i[0])
#                 print(tokenizations.get_original_spans(tokens, requirement_text))
                count+=1
    return count/total_sentences
#     print("Worliness index :"+"{:.2f}".format(count/total_sentences))

def get_ambiguity_index(sentences):
    global len_sent
    chunkGram = """
                    analytical: 
                        {<JJ><NN.*><NN.*>}
                        
                    coordination: 
                        {<JJ><NN.*><CC><NN.*>}
                        
                    PPAttachment: 
                        {<VB.*><DT>?<JJ>*<NN.*><IN><DT>?<JJ>*<NN.*>}
            """
    count = 0
    for sent in sentences:
        words = nltk.word_tokenize(sent)
        tagged = nltk.pos_tag(words)
        chunkParser = nltk.RegexpParser(chunkGram)
        result = chunkParser.parse(tagged)
        
        chunked_terms = []
        for e in result.subtrees(filter=lambda t: t.label() == 'analytical' or t.label() == 'coordination' or t.label() == 'PPAttachment'):
            if isinstance(e, tuple):
                chunked_terms.append([ e[0] ])
            else:
                chunked_terms.append([w for w, t in e])
                count+=1
                print(chunked_terms)
    return count/len_sent
#     print("Ambiguity index :"+"{:.2f}".format(count/len(sentences)))
        

def get_vagueness_index(sentences):
    dictionary = []
    global len_sent
    with open("Vagueness_Dictionary") as f:
        dictionary = f.read().splitlines()
    count = 0
    
    for sent in sentences:
        for token in dictionary:
            if token in sent:
                print(token)
                count+=1
    #modification required for considering POS tags
    return count/len_sent
#     print("Vagueness index :"+"{:.2f}".format(count/len(sentences)))

def get_readability_index(text):
    
    nc = len(text) 
    words = word_tokenize(text)
    nw = len(words)
    sentences = sent_tokenize(text)
    ns = len(sentences)
    
    #NC is the number of character in the text, NW the number of words, and NS the number of sentences
    ig = 0
    if not((nc == 0 and nw == 0) or (ns == 0 and nw == 0)):
        ig =  89 - 10*nc/nw + 300*ns/nw         #Gulpease Index IG
    
    chunkGram = r"""NP: 
                        {<DT>?<JJ>*<NN>+}
                        
                    VP: 
                        {<PRP>?<VB|VBD|VBZ|VBG>*<RB|RBR>?}
                    
                    ADJP:
                        {<RB|RBR>?<JJ>+}
                    
                    ABVP:
                        {<RB|RBR><RB|RBR>+}
                    
                    PP:
                        {<IN><DT>*<NN>}
    """
    nk = 0         #number of chunks
    
    #initializing each chunk phrase counter
    NP_count = 0
    VP_count = 0
    ADVP_count = 0
    ADJP_count = 0
    PP_count = 0
    
    for sent in sentences:
        tokens = nltk.word_tokenize(sent)
        tagged = nltk.pos_tag(tokens)
        chunkParser = nltk.RegexpParser(chunkGram)
        result = chunkParser.parse(tagged)
        
        NP_count += len(list(result.subtrees(filter=lambda t: t.label() == 'NP')))
        VP_count += len(list(result.subtrees(filter=lambda t: t.label() == 'VP')))
        ADVP_count += len(list(result.subtrees(filter=lambda t: t.label() == 'ADVP')))
        ADJP_count += len(list(result.subtrees(filter=lambda t: t.label() == 'ADJP')))
        PP_count += len(list(result.subtrees(filter=lambda t: t.label() == 'PP')))
        
        nk += (NP_count+VP_count+ADVP_count+ADJP_count+PP_count)
                       
    ik = 0        #Chunk Index Ik
    if nk > 1:
        ik = 100/(nk/len(sentences) - 1)
    else:
        ik = 100
    
    it = 0        #Chunk Type Index IT
    
    weighted_count = (NP_count*0.2691 + VP_count*0.4454 + ADJP_count*0.0379 + ADVP_count*0.0317 + PP_count*0.2159)
    
    if nk > 1:
        it = 544*weighted_count/nk
    else:
        it = 100*weighted_count/0.2468
    
    print("Ig :",ig,"Ik :",ik,"It :",it)
    
    readability_index = (ig + ik + it)/3
    return 1 - readability_index/100
#     print("Complexity index :"+"{:.2f}".format(1 - readability_index/100))
    
          



def get_understandability_index(text):
    all_words = word_tokenize(text)
    words = [w for w in all_words if w not in stop_words]
#     ps = PorterStemmer()
#     for i in range(len(words)):
#         words[i] = ps.stem(words[i])
    
    word_dictionary = []
    with open("Basic_english_words") as f:
        dictionary = f.read().splitlines()
    nl = 0          #least used words
    nb = len(all_words)-len(words)          #basic words
    for word in words:
        if word in word_dictionary:
            nb+=1
        else:
            nl+=1
    print("Nb:",nb,"Nl:",nl)
    iu = 0
    if not(nb == 0 and nl == 0):
        iu = (nb + 0.5*nl)/len(all_words)

    
    return 1-iu

def get_quality_metrics(text_str):
#     text_str = read_file("Point Of Sale System-PS.txt")
    global len_sent
    sentences = sent_tokenize(text_str)
    if len_sent == 0:
        len_sent = len(sentences)
    for i in range(len(sentences)):
        sentences[i] = "".join([j.lower() for j in sentences[i] if j not in string.punctuation])

    result_metrics = []
    #calling metrics functions
    print("-------------Duplication Index-----------")
    result_metrics.append(get_duplication_index(sentences))
    print("-------------Wordliness Index-----------")
    result_metrics.append(get_wordliness_index(text_str,len_sent))
    print("-------------Ambiguity Index-----------")
    result_metrics.append(get_ambiguity_index(sentences))
    print("-------------Vagueness Index-----------")
    result_metrics.append(get_vagueness_index(sentences))
    print("-------------Readability Index-----------")
    result_metrics.append(get_readability_index(text_str))
    print("-------------Understandability Index-----------")
    result_metrics.append(get_understandability_index(text_str))
    
    return result_metrics

In [22]:
from tkinter import *

def go(event,start_char,end_char):
    cs = outputtext.curselection()
    tbox2.tag_add("highlight", "1.0+"+str(start_char[cs[0]])+"c", "1.0+"+str(end_char[cs[0]])+"c") 
    tbox2.tag_config("highlight", background="yellow", foreground="black")
    
def getTextInput():
    desc=tbox1.get("1.0","end")
    req=tbox2.get("1.0","end")
    template_list, start_char, end_char = convert_requirements(desc,req)
    template_string = ""
    print("template list : " , template_list)
    for i in range(len(template_list)):
        template = template_list[i] if template_list[i] is not None else ''  # Check if template_list[i] is None
        outputtext.insert(i, str(i + 1) + ". " + str(template) + "\n\n")
        if template:
            template_string += template
    
    outputtext.bind('<Double-1>', lambda event, arg1=start_char, arg2 = end_char: go(event, arg1, arg2))
    
    
    text_str= desc + req
    output_metrics = get_quality_metrics(text_str)
    output_string = "Duplication index: "+"{:.2f}".format(output_metrics[0])+"\n"+"Wordliness index: "+"{:.2f}".format(output_metrics[1])+"\n"+"Ambiguity index: "+"{:.2f}".format(output_metrics[2])+"\n"+"Vagueness index: "+"{:.2f}".format(output_metrics[3])+"\n"+"Redability index: "+"{:.2f}".format(output_metrics[4])+"\n"+"Understandability index: "+"{:.2f}".format(output_metrics[5])
    outputtext1.insert(END, output_string)
    
    print(template_string)
    
    output_metrics1 = get_quality_metrics(template_string)
    output_string1 = "Duplication index: "+"{:.2f}".format(output_metrics1[0])+"\n"+"Wordliness index: "+"{:.2f}".format(output_metrics1[1])+"\n"+"Ambiguity index: "+"{:.2f}".format(output_metrics1[2])+"\n"+"Vagueness index: "+"{:.2f}".format(output_metrics1[3])+"\n"+"Redability index: "+"{:.2f}".format(output_metrics1[4])+"\n"+"Understandability index: "+"{:.2f}".format(output_metrics1[5])
    outputtext2.insert(END, output_string1)
    
    # for i in range(len(template_list)):
    #     template_string += str(i+1) + ". " + template_list[i] + "\n\n"
    # outputtext.insert(END, template_string)

root = Tk()
root.title("Automated conversion to EARS/Rupp's template")
frame=Frame(root, width=1300, height=600)
frame.pack()

len_sent = 0

text1 = Label(text="Requirement description:")
text1.place(x=10, y=10)

tbox1 = Text(frame,font=('Times', 14))
tbox1.place(x=10, y=40, height=150, width=300)

text2 = Label(text="Requirement Textual Specification:")
text2.place(x=10, y=200)

tbox2 = Text(frame,font=('Times', 14))
tbox2.place(x=10, y=225, height=360, width=300)

button1 = Button(frame, text="Convert Requirements", command = getTextInput)
button1.place(x=320, y=300, height=50, width=125)



# outputtext = Text(frame)
# outputtext.place(x=450, y = 40, height=540, width=500)

text3 = Label(text="Converted Requirements to EARS/Rupp's template format:")
text3.place(x=450, y=10)

outputtext = Listbox(frame,height=10, font=('Times', 14))
outputtext.place(x=450, y = 40, height=360, width=800)

text4 = Label(text="Quality metrics before conversion:")
text4.place(x=450, y=420)

# outputtext1 = Listbox(frame,height=10, font=('Times', 14))
outputtext1 = Text(frame,height=10, font=('Times', 14))
outputtext1.place(x=450, y = 450, height=130, width=350)

text5 = Label(text="Quality metrics after conversion:")
text5.place(x=900, y=420)

# outputtext1 = Listbox(frame,height=10, font=('Times', 14))
outputtext2 = Text(frame,height=10, font=('Times', 14))
outputtext2.place(x=900, y = 450, height=130, width=350)



root.mainloop()

---------------ACTORS--------------------
1 : Library Management System

2 : user

3 : Library Management System

4 : system

5 : details

6 : user

7 : user

8 : system

9 : user

10 : user

11 : librarian

12 : user

13 : system

14 : member

15 : librarian

---------------PRE-CONDITION--------------------
1 : Member must be logged in to the system.

2 : The books should be stored in database.

3 : The books must be ready to retrieve.

4 : If User does not want to print the details then user can ignore the step.

5 : User can reserve a book by inputting the relevant details & the librarian can also reserve a book for a member.

6 : User should be logged into the system.

7 : User should have correct book Id. Books should be available to reserve.

8 : Members should be logged into the system.

9 : Guest user can also search books.

10 : Book should be available to search.

11 : Member should give the member Id to the librarian.

12 : Books should be available to issue.

13 : The libra

Exception in Tkinter callback
Traceback (most recent call last):
  File "c:\Users\u25o52\AppData\Local\Programs\Python\Python39\lib\tkinter\__init__.py", line 1885, in __call__
    return self.func(*args)
  File "C:\Users\u25o52\AppData\Local\Temp\ipykernel_8832\173835817.py", line 11, in getTextInput
    template_list, start_char, end_char = convert_requirements(desc,req)
  File "C:\Users\u25o52\AppData\Local\Temp\ipykernel_8832\2383217313.py", line 101, in convert_requirements
    res = make_Rupp_template_3(sent,Rupp_template_3,actor)
  File "C:\Users\u25o52\AppData\Local\Temp\ipykernel_8832\1024907574.py", line 19, in make_Rupp_template_3
    return correct_grammar("".join(temp))
  File "C:\Users\u25o52\AppData\Local\Temp\ipykernel_8832\2442092847.py", line 8, in correct_grammar
    corrected_text = tool.correct(string)
  File "c:\Users\u25o52\AppData\Local\Programs\Python\Python39\lib\site-packages\language_tool_python\server.py", line 153, in correct
    return correct(text, self.